In [159]:
from openai import OpenAI
from dotenv import load_dotenv

import json
import os
import pandas as pd
from tqdm import tqdm
import re
from typing import Dict

tqdm.pandas()

In [160]:
load_dotenv()

INFERENCE_MODEL = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
DEEPINFRA_TOKEN = os.environ.get('DEEPINFRA_API_TOKEN')

In [161]:
rools = '\n'.join(map(str, json.load(open('data/resume_rools.json'))))

In [162]:
prompt = """
Ты — ИИ-модератор резюме. Проверь текст резюме на соответствие следующим правилам. Если есть нарушения, верни JSON-объект с типом нарушения и фрагментом текста, который нарушает правило. Если нарушений нет, верни статус "OK".

**Правила:** 
{rools}

**Инструкции:**

- Тщательно проанализируй каждое предложение в резюме.
- Если нарушений нет, верни "status": "OK" и пустой массив violated_rules.
- Если фрагмент нарушает правило, укажи его точную цитату в resume_fragment.
- Ответ должен содержать твои рассуждения по каждому правилу оканчивающийся вердиктом, затем должен быть результат в формате JSON, без Markdown и комментариев.

Формат ответа: Рассуждения, Результат: { "status": "OK" | "violation", "violated_rules": [] | [ { "id": "rule_id", "condition": "Текст условия правила на русском", "resume_fragment": "Точная цитата из резюме" } ] }

**Текст резюме:**
{resume_text}
"""

In [163]:
print(prompt.replace('{rools}', rools))


Ты — ИИ-модератор резюме. Проверь текст резюме на соответствие следующим правилам. Если есть нарушения, верни JSON-объект с типом нарушения и фрагментом текста, который нарушает правило. Если нарушений нет, верни статус "OK".

**Правила:** 
{'id': 'rule_1', 'condition': 'Резюме должно содержать информацию о трудовой деятельности, однако оно может не содержать, если желаемая должность не требует опыта'}
{'id': 'rule_2', 'condition': 'Резюме должно содержать профессию/должность или должности, на которые претендует соискатель, должность должжна быть реальной профессией'}
{'id': 'rule_4', 'condition': 'Резюме должно быть заполнено на русском языке'}
{'id': 'rule_5', 'condition': 'Информация о работодателе не должна содержать сведений, не относящихся к работодателю, если работодатель ИП это не нарушение правил'}
{'id': 'rule_6', 'condition': 'Информация о резюме не должна содержать сведений, не относящихся к соискателю'}
{'id': 'rule_7', 'condition': 'Информация не должна содержать слова и

In [164]:
def process_answer(text: str) -> Dict[str, str]:
    result_match = re.search(r'Результат:\s*(.*)', text, re.DOTALL)
    if result_match:
        result = json.loads(result_match.group(1).strip())
    else:
        result = None

    discussion_match = re.search(r'Рассуждения:\s*(.*?)\s*\n\nРезультат:', text, re.DOTALL)

    if discussion_match:
        discussion_text = discussion_match.group(1).strip()
    else:
        discussion_text = None
    
    return {'reasoning': discussion_text, 'result': result}

In [165]:
import requests
from typing import Dict

def get_answer(resume_text: str) -> Dict[str, str]:
    url = "https://api.deepinfra.com/v1/openai/chat/completions"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {DEEPINFRA_TOKEN}"
    }
    
    formatted_prompt = prompt.replace('{rools}', rools).replace('{resume_text}', resume_text)
    
    data = {
        "model": INFERENCE_MODEL,
        "messages": [
            {
                "role": "user",
                "content": formatted_prompt
            }
        ],
        "temperature": 0,
        "seed": 42
    }
    
    response = requests.post(url, json=data, headers=headers)
    response.raise_for_status()
    
    answer_content = response.json()['choices'][0]['message']['content']
    
    return process_answer(answer_content)

In [166]:
df = pd.read_csv('./data/processed_resume.csv')

In [ ]:
llama_answers = df['text_data'].progress_apply(get_answer).tolist()

  2%|▏         | 20/905 [03:06<2:29:47, 10.15s/it]

In [ ]:
df['llama_answers'] = llama_answers

{'reasoning': '- Правило rule_1: Резюме содержит информацию о трудовой деятельности, поэтому правило не нарушено.\n- Правило rule_2: Резюме содержит профессию/должность, на которую претендует соискатель, поэтому правило не нарушено.\n- Правило rule_4: Резюме заполнено на русском языке, поэтому правило не нарушено.\n- Правило rule_5: Информация о работодателе не содержит сведений, не относящихся к работодателю, поэтому правило не нарушено.\n- Правило rule_6: Информация о резюме не содержит сведений, не относящихся к соискателю, поэтому правило не нарушено.\n- Правило rule_7: Информация не содержит слова и выражения, не соответствующие нормам современного русского литературного языка, поэтому правило не нарушено.',
 'result': {'status': 'OK', 'violated_rules': []}}